In [2]:
#Konstantinos Mavromatis

import numpy as np
from string import *
from Bio import  SeqIO
import sys 



#Read sequences
in_seq_handle = open("hw3.fna")

seq_dict = SeqIO.to_dict(SeqIO.parse(in_seq_handle, "fasta"))
in_seq_handle.close()
seq_dict_keys =  seq_dict.keys()

N = len(seq_dict.values())



distance_seqs = np.zeros((N,N))
#print(seq_dict.values())
seqs_list= []
seqs_id = []
#i = 0
for sequence in seq_dict.values():
    seq1 = sequence.seq
    seqs_list.append(str(seq1))
    seqs_id.append(sequence.id)

    


In [3]:
#Create distance matrix
for i in range(N):
    s1 = seqs_list[i]
    distance_seqs[i,i] = 0
    for j in range(i+1,N):
        s2 = seqs_list[j]
        dist = 0
        for k in range(len(s1)):
            if s1[k]!=s2[k]:
                dist+=1
        distance_seqs[i,j] = dist/len(s1)
        distance_seqs[j,i] = dist/len(s1)
print(distance_seqs)
        
with open('distances.txt', 'w') as f:  
    f.write('\t')
    f.write('\t'.join(seqs_id) + '\n')
    for i in range(N):
        f.write(seqs_id[i]) 
        for j in range(N):
            f.write('\t'+str(distance_seqs[i,j]))
        f.write('\n')
    


[[0.         0.24024226 0.26850606 ... 0.46635262 0.44010767 0.42664872]
 [0.24024226 0.         0.27927322 ... 0.44952894 0.4858681  0.49461642]
 [0.26850606 0.27927322 0.         ... 0.46500673 0.46096904 0.45356662]
 ...
 [0.46635262 0.44952894 0.46500673 ... 0.         0.28263795 0.27523553]
 [0.44010767 0.4858681  0.46096904 ... 0.28263795 0.         0.18506057]
 [0.42664872 0.49461642 0.45356662 ... 0.27523553 0.18506057 0.        ]]


In [4]:
#Neighbor Joining Algorithm:Basics
import sys
import numpy as np
import scipy as scipy
import itertools
#from sys import maxint
maxint = 10000
#Compute Q matrix
def calculateQ(d):
    r = d.shape[0]
    q = np.zeros((r,r))
    for i in range(r):
        for j in range(r):
            if i == j:
                q[i][j] = 0
            else:
                sumI = 0
                sumJ = 0
                for k in range(r):
                    sumI += d[i][k]
                    sumJ += d[j][k]
                q[i][j] = (r-2) * d[i][j] - sumI - sumJ

    return q

#Finds the lowest distance
def findLowestPair(q):
    r = q.shape[0]
    minVal = maxint
    for i in range(0,r):
        for j in range(i,r):
            if (q[i][j] < minVal):
                minVal = q[i][j]
                minIndex = (i,j)
    return minIndex

#Merges two nodes into one new node
def doDistOfPairMembersToNewNode(i,j,d):
    r = d.shape[0]
    sumI = 0
    sumJ = 0
    for k in range(r):
        sumI += d[i][k]
        sumJ += d[j][k]

    dfu =  (1. / (2. * (r - 2.))) * ((r - 2.) * d[i][j] + sumI - sumJ)
    dgu = d[i][j] - dfu#(1. / (2. * (r - 2.))) * ((r - 2.) * d[i][j] - sumI + sumJ)

    return (dfu,dgu)

#Recreates the distance matrix based on the merged nodes
def calculateNewDistanceMatrix(f,g,d):
    #print(d)
    r = d.shape[0]
    nd = np.zeros((r-1,r-1))

    # Copy over the old data to this matrix
    ii = jj = 1
    for i in range(0,r):
        if i == f or i == g:
            continue
        for j in range(0,r):
            if j == f or j == g:
                continue
            nd[ii][jj] = d[i][j]
            jj += 1
        ii += 1
        jj = 1
            
    # Calculate the first row and column
    ii = 1
    for i in range(0,r):
        if i == f or i == g:
            continue
        nd[0][ii] = (d[f][i] + d[g][i] - d[f][g]) / 2.
        nd[ii][0] = (d[f][i] + d[g][i] - d[f][g]) / 2.
        ii += 1

    return nd
    

In [5]:
#Creates edges.txt
def doNeighbourJoining_Edges(d, labels, newlabel):
     
    #Neighbor joining    
    q = calculateQ(d)
    lowestPair = findLowestPair(q)
    i = lowestPair[0]
    j = lowestPair[1]
    pairDist = doDistOfPairMembersToNewNode(i,j,d)
    
    print(newlabel, labels[i], pairDist[0])
    print(newlabel, labels[j], pairDist[1])
    
    #new distances
    d = calculateNewDistanceMatrix(i,j,d)

    #add new nodes
    f.write(str(newlabel)+'\t'+str(labels[i])+'\t'+str(pairDist[0])+'\n')
    f.write(str(newlabel)+'\t'+str(labels[j])+'\t'+str(pairDist[1])+'\n')

    #delete merged nodes
    del labels[lowestPair[0]]
    if lowestPair[0] < lowestPair[1]:
        del labels[lowestPair[1]-1]
    else:
        del labels[lowestPair[1]]

    #add new node
    labels.insert(0,newlabel)

    #Pre-order
    newlabel -= 1

    #add last two nodes
    if d.shape[0] == 2:
        f.write(str(labels[0])+'\t'+str(labels[1])+'\t'+str(pairDist[0])+'\n')
        print(labels[0], labels[1], pairDist[0])

    #else recursion
    else:
        doNeighbourJoining_Edges(d, labels, newlabel)
    

In [6]:
#Tips are labelled from 1 to 61
labels = [*range(1,N+1)]

#Root is 62
newlabel = 2*N-2

with open('edges.txt', 'w') as f:  
    doNeighbourJoining_Edges(distance_seqs, labels, newlabel)

120 1 0.11558728927618218
120 2 0.12465497182745174
119 120 0.021519817143917916
119 3 0.13224868891260966
118 16 0.04764645462917049
118 33 0.033780194092229245
117 9 0.08126592241876557
117 59 0.0762037949432802
116 30 0.03437232350422126
116 60 0.026192952404257883
115 10 0.0614095757938288
115 57 0.05433739594237577
114 25 0.022691345641077738
114 29 0.020377295004951872
113 114 0.0005653729682161525
113 23 0.021978368619939976
112 116 0.013572757501385457
112 35 0.04867488718232921
111 113 0.010918993943472373
111 28 0.0380379374158816
110 117 0.025517500068668077
110 50 0.08249057530145304
109 27 0.06864677966576939
109 40 0.09420651777703008
108 17 0.03525367973425734
108 18 0.03136812376506972
107 109 0.005103337673357104
107 5 0.08910931373983266
106 34 0.041981643487363565
106 46 0.03877205772394195
105 110 0.017747317768873103
105 115 0.030284310764101326
104 13 0.0802724654918777
104 51 0.08258083195092177
103 106 0.01123732214958662
103 37 0.045963216208421455
102 104 0.00

In [7]:
#Creates tree.txt 
def doNeighbourJoining_Tree(d, labels):
    
    #Neighbor joining
    q = calculateQ(d)
    lowestPair = findLowestPair(q)
    i = lowestPair[0]
    j = lowestPair[1]
    pairDist = doDistOfPairMembersToNewNode(i,j,d)

    #Post-order
    newlabel  = '('+str(labels[i])+':'+str(pairDist[0])+','+str(labels[j])+':'+str(pairDist[1])+')'

    #print(newlabel)
    
    #new distances
    d = calculateNewDistanceMatrix(i,j,d)

    #new labels, deleted merged, add new node
    del labels[lowestPair[0]]
    if lowestPair[0] < lowestPair[1]:
        del labels[lowestPair[1]-1]
    else:
        del labels[lowestPair[1]]
    labels.insert(0,newlabel)

    #finished, last two
    if d.shape[0] == 2:
        newlabel  = '('+str(labels[0])+':'+str(d[0,1])+','+str(labels[1])+':'+str(d[1,0])+')'
        f.write(newlabel+';')
        print(newlabel+';')
    #else recursion
    else:
        doNeighbourJoining_Tree(d, labels)
            
        #print(labels,d)
        
    

In [8]:

with open('tree.txt', 'w') as f:  
    #labels are IDs now
    doNeighbourJoining_Tree(distance_seqs, seqs_id.copy())

((((((536135:0.09131794024319574,2678325:0.09576146756299538):0.0024041526250210277,990269:0.0689282834449655):0.01149387967164018,((4314491:0.09547367693712747,4472218:0.05728540785425881):0.0028196359127497647,847142:0.07019516893247231):0.01382577039565458):0.003768279125680835,(698164:0.0596647814497813,3183891:0.06483050791764805):0.009648611856822522):0.0009590806101084058,(((((((106543:0.041981643487363565,3148512:0.03877205772394195):0.01123732214958662,5364:0.045963216208421455):0.024851971263038355,11136:0.1041857137975269):0.0038911173651250963,((((593544:0.060210401244952895,779409:0.04678825286002692):0.006977567658506386,4387271:0.043157021843512444):0.016551387165629216,286686:0.08674605563383243):0.003386822099463411,(((166838:0.03437232350422126,4468812:0.026192952404257883):0.013572757501385457,100149:0.04867488718232921):0.03129593309485016,348167:0.08932990808953747):0.010787134831895927):0.011707595622761843):0.0038721042665573143,((((260976:0.08221338144106904,646

In [9]:
#Help me create boot.txt. Keep track of tips. (no recursion here)
def doNeighbourJoining_Boot(d, labels):
    labels = []
    for x in range(1,N+1):
        labels.append([x])
    
    nodes = []
    
    while d.shape[0] > 2:
        q = calculateQ(d)
        lowestPair = findLowestPair(q)
        i = lowestPair[0]
        j = lowestPair[1]
        pairDist = doDistOfPairMembersToNewNode(i,j,d)
        d = calculateNewDistanceMatrix(i,j,d)
        
        #merge tips partitioned
        newlabel = labels[i] + labels[j]
        nodes.append(newlabel)
        
        del labels[lowestPair[0]]
        if lowestPair[0] < lowestPair[1]:
            del labels[lowestPair[1]-1]
        else:
            del labels[lowestPair[1]]
        labels.insert(0,newlabel)

        
    nodes.append(labels[0] + labels[1])
    return nodes
        

In [10]:
nodes = doNeighbourJoining_Boot(distance_seqs, seqs_id)
for j in range(len(nodes)):
    nodes[j].sort()
print(nodes, len(nodes))

[[1, 2], [1, 2, 3], [16, 33], [9, 59], [30, 60], [10, 57], [25, 29], [23, 25, 29], [30, 35, 60], [23, 25, 28, 29], [9, 50, 59], [27, 40], [17, 18], [5, 27, 40], [34, 46], [9, 10, 50, 57, 59], [13, 51], [34, 37, 46], [9, 10, 13, 50, 51, 57, 59], [14, 44], [31, 32], [22, 30, 35, 60], [14, 44, 55], [7, 8], [20, 54], [21, 23, 25, 28, 29], [16, 33, 52], [7, 8, 16, 33, 52], [4, 7, 8, 16, 33, 52], [5, 9, 10, 13, 27, 40, 50, 51, 57, 59], [4, 7, 8, 11, 16, 33, 52], [26, 49], [20, 21, 23, 25, 28, 29, 54], [15, 48], [4, 7, 8, 11, 16, 20, 21, 23, 25, 28, 29, 33, 52, 54], [4, 7, 8, 11, 16, 20, 21, 23, 25, 28, 29, 33, 52, 54, 56], [26, 39, 49], [1, 2, 3, 5, 9, 10, 13, 27, 40, 50, 51, 57, 59], [15, 45, 48], [14, 24, 44, 55], [14, 22, 24, 30, 35, 44, 55, 60], [4, 7, 8, 11, 16, 20, 21, 23, 25, 26, 28, 29, 33, 39, 49, 52, 54, 56], [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 16, 20, 21, 23, 25, 26, 27, 28, 29, 33, 39, 40, 49, 50, 51, 52, 54, 56, 57, 59], [15, 41, 45, 48], [15, 17, 18, 41, 45, 48], [53, 61], [43

In [ ]:
#initialize confidence to zero
conf = []
for j in range(len(nodes)):
    nodes[j].sort()
    conf.append(0)
    
    
from random import randint

#Start loop
runs = 100
for _ in range(runs):
    distance_boot = np.zeros((N,N))
    new_seq = []
    
    #Bootstrap
    for i in range(len(seqs_list)):
        new_seq.append("")
    for k in range(len(seqs_list[0])):
        col = randint(0, len(seqs_list[0])-1)
        for i in range(len(seqs_list)):
            new_seq[i] = new_seq[i]+seqs_list[i][col]
            
    #New 'fake' distances
    for i in range(N):
        s1 = new_seq[i]
        distance_boot[i,i] = 0
        for j in range(i+1,N):
            s2 = new_seq[j]
            dist = 0
            for k in range(len(s1)):
                if s1[k]!=s2[k]:
                    dist+=1
            distance_boot[i,j] = dist/len(s1)
            distance_boot[j,i] = dist/len(s1)
            
    #Repeat, bulding the new tree
    nodes_boot = doNeighbourJoining_Boot(distance_boot, seqs_id)
    
    #Update confidences
    for j in range(len(nodes)):
        nodes_boot[j].sort()
    for j in range(len(nodes)):
        for k in range(len(nodes)):
            if (nodes[j])==(nodes_boot[k]):
                conf[j] +=1
    print(conf)

with open('boots.txt', 'w') as f:  
    for j in range(len(conf)):
        f.write(str(conf[j]/runs)+"\n")
    

[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1]
[2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 0, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 2]
[3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 1, 3, 1, 3, 1, 2, 3, 0, 3, 3, 2, 1, 2, 3, 3, 1, 3, 3, 3, 1, 3, 2, 0, 1, 1, 0, 2, 0, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 3]
[4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 2, 4, 2, 4, 2, 3, 4, 1, 3, 4, 3, 2, 2, 4, 4, 2, 3, 4, 4, 2, 4, 3, 1, 2, 2, 0, 2, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 1, 2, 3, 3, 1, 0, 0, 0, 0, 0, 0, 4]
[5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 2, 2, 5, 2, 5, 2, 4, 5, 1, 4, 5, 4, 2, 2, 5, 5, 2, 3, 5, 5, 2, 5, 3, 2, 2, 2, 0, 2, 2, 1, 1, 0, 0, 3, 3, 2, 3, 3, 2, 3, 4, 4, 1, 1, 1, 1, 0, 1, 0, 5]
[6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 3, 3, 6, 3, 6, 2, 5, 6, 2, 5, 6, 5, 2, 2, 6, 6, 2, 3, 6, 5, 2, 6